In [13]:
from __future__ import division
import cv2
import numpy as np

# Create kernels for morphological operations
kernelOpen = np.ones((5, 5), np.uint8)
kernelClose = np.ones((20, 20), np.uint8)

# Set the path to your image here
image_path = 'raw_banana.jpg'

# Load the image
image = cv2.imread(image_path)
if image is None:
    raise ValueError("Image not found or unable to read.")

# Convert BGR to HSV
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Define color ranges and create masks
masks = {
    'red1': cv2.inRange(hsv, np.array([0, 50, 50]), np.array([10, 255, 255])),
    'red2': cv2.inRange(hsv, np.array([170, 50, 50]), np.array([180, 255, 255])),
    'green': cv2.inRange(hsv, np.array([40, 40, 40]), np.array([70, 255, 255])),
    'yellow': cv2.inRange(hsv, np.array([20, 50, 50]), np.array([30, 255, 255]))
}
masks['red'] = masks['red1'] + masks['red2']

# Calculate color percentages
total_pixels = hsv.size / 3  # Number of pixels per channel
color_counts = {color: np.sum(mask == 255) for color, mask in masks.items()}
color_percents = {color: count / total_pixels * 100 for color, count in color_counts.items()}

# Determine ripeness based on dominant color proportion
ripeness = "Unknown"

# Find the dominant color by finding the maximum percentage
dominant_color = max(color_percents, key=color_percents.get)

if dominant_color == 'green':
    ripeness = "Low Ripeness"
elif dominant_color == 'yellow':
    ripeness = "Medium Ripeness"
elif dominant_color == 'red':
    ripeness = "High Ripeness"

# Print color percentages and ripeness in the console
print(f"Ripeness: {ripeness}")
print(f"Red: {color_percents['red']:.2f}%")
print(f"Green: {color_percents['green']:.2f}%")
print(f"Yellow: {color_percents['yellow']:.2f}%")

# Find the largest contour in the combined mask
total_mask = sum(masks.values())
contours, _ = cv2.findContours(total_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
if contours:
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.putText(image, ripeness, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)

# Show the result
cv2.imshow('Image with Ripeness', image)

# Wait for the ESC key
while True:
    if cv2.waitKey(5) == 27:  # ESC key
        break

# Clean up
cv2.destroyAllWindows()

Ripeness: Low Ripeness
Red: 0.03%
Green: 7.67%
Yellow: 0.30%
